In [1]:
from selenium import webdriver 
import pandas as pd 
from selenium.webdriver.common.by import By 
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC
import time, re
import pymongo
from pymongo import MongoClient

webdriverpath = "/home/eesha/Softwares/chromedriver_linux64/chromedriver"
url = "https://timesofindia.indiatimes.com/india/sena-never-stopped-talkswill-adhere-to-coalition-dharma-sanjay-raut/articleshow/71864810.cms"

In [2]:
driver = webdriver.Chrome(webdriverpath)
driver.maximize_window()

In [3]:
client = MongoClient()
db = client['CodeMixed-Data-Test']
collection = db['TOI-Comments']

In [4]:
def scrape(url):
    all_comments = []

    driver.get(url)
    
    article_name = driver.find_element_by_xpath('//*[@id="app"]/div/div[3]/div[1]/div[2]/div[2]/h1').text
    article = driver.find_element_by_xpath('//*[@id="app"]/div/div[3]/div[1]/div[2]/div[5]').text

    xpath = '//*[@id="app"]/div/div[3]/div[1]/div[2]/div[5]/div[5]/div/div[2]'
    button = driver.find_element_by_xpath(xpath)
    button.click()
    
    CLICK_PAUSE_TIME = 2

    while True:
        try:
            more_comments_button = driver.find_elements_by_xpath('//*[@id="comment-sidebar-71864810"]/div/div[3]/div[1]/div[2]/div[1]/div/div/div[2]')[0]
            more_comments_button.click()
            time.sleep(CLICK_PAUSE_TIME)
        except:
            break
            
    comments = driver.find_elements_by_xpath('//div[@class="_2uauV"]')
    likes = driver.find_elements_by_xpath('//span[@class="_23uA0"]')
    dislikes = driver.find_elements_by_xpath('//span[@class="_20ocg"]')

    for comment, like, dislike in zip(comments, likes, dislikes):
        if comment.text != '':
            temp = {'article_name': article_name, 'comment': comment.text, 'likes': like.text, 'dislikes': dislike.text}
            all_comments.append(temp)
            
    data = {'title': article_name, 'article': article, 'comments': all_comments}
    return data

In [5]:
def save_to_csv(filename, data):
    import csv
    with open('toi_comments_test.csv', 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["Comment", "Likes", "Dislikes"])
        for comment, likes_count, dislikes_count in zip(comment_body, like_count, dislike_count):
            if 'K' in likes_count:
                likes_count = likes_count.lower().split('k')[0]
                likes_count = likes_count + '000'
            writer.writerow([comment, likes_count, dislikes_count])

In [11]:
def save_to_mongo(data):
    collection.insert_one(data)

In [7]:
data = scrape(url)

In [8]:
print(len(data))

3


In [9]:
print(data)

{'title': 'Sena never stopped talks, will adhere to coalition dharma: Sanjay Raut', 'article': 'MUMBAI: Amid speculations regarding probable political permutations and combinations vis-a-vis formation of the next government in Maharashtra, the Shiv Sena on Saturday said it would adhere to the "coalition dharma", a remark being viewed as softening of its stand towards the BJP.\nThe Sena and the BJP are locked in a battle of nerves over sharing of power.\nMaharashtra power tussle: Sonia Gandhi, Sharad Pawar to discuss backing Shiv Sena\n\nBoth the parties, who were partners in the outgoing government, have not even started formal discussions even though the tenure of the current Legislative Assembly ends on November 8.\nNEWS IN BRIEF\nEXPLORE BRIEFS\nWith OPDs, OTs shut, many cancer, HIV patients hit hard\nWith entire healthcare set-up fighting Covid-19, patients of diseases like HIV, cancer and renal disease who need to access hospitals regularly, and are immuno-compromised, are facing 

In [12]:
save_to_mongo(data)

In [13]:
driver.close()